#Collect API Data from Deadlock-API Project, filter, format, and insert into Deadlock.db

#Requirements import from .txt
#Fetch data based on URL (site+endpoint)
##Fetch hero data
#Filter fetched data based on filters
##Filter hero data
#prints or formatting

#Requirements import from .txt

In [ ]:
!pip install -r requirements.txt

In [1]:
import requests
import pandas as pd
import numpy as np

#Fetch data

## Fetch hero data, function returns a df

In [2]:
#Fetches data and returns DF
def fetch_hero_data():
    
    site = "https://assets.deadlock-api.com"
    endpoint = "/v2/heroes/"
    url = site+endpoint
    
    heroes_df = fetch_hero_data(url)

    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        heroes_data = response.json()  # Converts the JSON response to a Python dictionary
    else:
        print(f"Failed to retrieve hero data: {response.status_code}")
    
    df = pd.DataFrame(heroes_data)
    return df

##Fetch Match data, function returns a DF

In [3]:

#Fetches data and returns DF
def fetch_match_data():
    
    site = "https://api.deadlock-api.com"
    endpoint = "/v1/matches/active"
    url = site+endpoint

    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        active_matches_data = response.json()  # Converts the JSON response to a Python dictionary
    else:
        print(f"Failed to retrieve match data: {response.status_code}")
    
    df = pd.DataFrame(active_matches_data)
    return df

In [29]:
active_matches_df = fetch_match_data()
print(active_matches_df.columns.tolist())

['start_time', 'winning_team', 'winning_team_parsed', 'match_id', 'players', 'lobby_id', 'game_mode_version', 'net_worth_team_0', 'net_worth_team_1', 'duration_s', 'spectators', 'open_spectator_slots', 'objectives_mask_team0', 'objectives_mask_team1', 'match_mode', 'match_mode_parsed', 'game_mode', 'match_score', 'region_mode', 'region_mode_parsed', 'compat_version']


In [ ]:
row_dict = active_matches_df.iloc[0].to_dict()
print(row_dict)

In [ ]:
print(active_matches_df.dtypes)

## Filter match data - IN PROGRESS, BUILDING LOGIC TO HANDLE PLAYER DICT.

In [31]:
def process_active_matches(active_matches_df):
    match_players = []

    for index, row in active_matches_df.iterrows():
        #print(f"row data: {row}")
        # Iterate through each column (key-value pair) of the row
        for key, value in row.items():
            if isinstance(value, list):  # If the value is a list (e.g., player_ids or hero_ids)
                print(f"Processing list in column: {key}")
                for item in value:
                    # Process each item in the list
                    print(f"Item: {item}")
                    match_players.append({
                        "match_id": row["match_id"],
                        "player_id": item  # Example of adding individual list items
                    })
            else:  # If the value is not a list (e.g., match_id, game_mode)
                print(f"Processing single value in column: {key} - Value: {value}")
                match_players.append({
                    "match_id": row["match_id"],
                    "game_mode": row["game_mode"],
                    "start_time": row["start_time"]
                })

    return pd.DataFrame(match_players)

# Call the function
match_players_df = process_active_matches(active_matches_df)
print(match_players_df)


Processing single value in column: start_time - Value: 1743384061
Processing single value in column: winning_team - Value: None
Processing single value in column: winning_team_parsed - Value: None
Processing single value in column: match_id - Value: 34444130
Processing list in column: players
Item: {'account_id': 1861952447, 'team': 0, 'team_parsed': 'Team0', 'abandoned': None, 'hero_id': 50}
Item: {'account_id': 112724001, 'team': 0, 'team_parsed': 'Team0', 'abandoned': None, 'hero_id': 2}
Item: {'account_id': 408158148, 'team': 0, 'team_parsed': 'Team0', 'abandoned': None, 'hero_id': 15}
Item: {'account_id': 201517885, 'team': 0, 'team_parsed': 'Team0', 'abandoned': None, 'hero_id': 14}
Item: {'account_id': 73143430, 'team': 0, 'team_parsed': 'Team0', 'abandoned': None, 'hero_id': 35}
Item: {'account_id': 74952111, 'team': 0, 'team_parsed': 'Team0', 'abandoned': None, 'hero_id': 4}
Item: {'account_id': 435212386, 'team': 1, 'team_parsed': 'Team1', 'abandoned': None, 'hero_id': 3}
Ite

In [15]:
print(active_matches_df[0])

0                                                      0
1      start_time                                    ...
2                                                      1
3      start_time                                    ...
4                                                      2
                             ...                        
395    start_time                                    ...
396                                                  198
397    start_time                                    ...
398                                                  199
399    start_time                                    ...
Name: 0, Length: 400, dtype: object


In [16]:
row_dict = active_matches_df.iloc[0].to_dict()
print(row_dict)

{0: 0}


In [13]:
print(active_matches_df.columns.tolist())

[0]


##Filter Data

In [ ]:
def filter_hero_data(df):\
    
    #columns to fetch
    filters = ["id", "classname", "name", "description", "player_selectable", "disabled", "starting_stats", "level_info", "scaling_stats", "standard_level_up_upgrades"]

    filtered_data = []
    for _, item in df.iterrows(): #Pulls each row of df
        filtered_item = {key: item[key] for key in filters if key in item} #build list of columns in df that matches variable filters
        filtered_data.append(filtered_item) #stores coulmns (and values) from df that match filters
    
    return pd.DataFrame(filtered_data)


##Format Data

In [ ]:
def format_hero_data(df):
    df.rename(columns={'id' : 'hero_id'}, inplace=True)
    return df

###Dispaly column headers.

In [ ]:
# Display all columns
pd.set_option('display.max_columns', None)
heroes_df.head()  # Now it'll show all columns

###print header

In [ ]:
#print(heroes_df.head(1))

### Convert fitlered hero data to json

In [ ]:
def heroes_to_json(df)
    heroes_json = df.to_dict(orient='records')
    return heroes_json

### insert Hero data to database function

In [ ]:

def insert_hero_data_to_db(df,db):
    import sqlite3

    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    for hero in df:
        cursor.execute("""
            INSERT INTO heroes (hero_id, name, player_selectable, disabled)
            VALUES (?, ?, ?, ?)
        """, (
                 hero['hero_id'], 
            hero['name'],
            hero['player_selectable'], 
            hero['disabled'], 
        ))
    conn.commit()
    print("Data was successfully loaded")


### insert starting stats into stats table

In [ ]:
def insert_starting_stats():
    import sqlite3
    
    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    
    for hero in heroes_json:
        hero_id = hero["hero_id"]
        stats = hero.get("starting_stats", {})
    
        for stat_name, stat_obj in stats.items():
            base = stat_obj.get("value", None)
            if base is not None:
                cursor.execute("""
                    INSERT INTO hero_stats (hero_id, stat_name, base_value)
                    VALUES (?, ?, ?)
                """, (hero_id, stat_name, base))
                print(f"Expected row insertion for: {base}")
    conn.commit()

### Insert scaling stats into stats table, matching hero / stat (Not working)

In [ ]:
def insert_scaling_stats():
    import sqlite3
    
    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    
    for hero in heroes_json:
        hero_id = hero["hero_id"]
        scaling_stats = hero.get("scaling_stats", {})
    
        for stat_name, scale_value in scaling_stats.items():
            cursor.execute("""
                UPDATE hero_stats
                SET scale_value = ?
                WHERE hero_id = ? AND stat_name = ?
            """, (scale_value, hero_id, stat_name))
    
    conn.commit()